# Unidad 03. Regresión lineal de una variable

El objetivo de esta práctica es comprender los fundamentos del entrenamiento de un modelo de ML simple. En particular, por su simplicidad, nos centraremos en un modelo de regresión lineal de una sola variable. 

En la vida real es muy improbable que tengáis que implementar modelos desde cero, ya que existe una gran variedad de librerías de ML disponibles, pero enfrentarse a la implementación de los conceptos más básicos os ayudará a entender cómo funcionan internamente los modelos y ver que no existe "magia" en el ML.

**Importante**: este Notebook esta desarrollado con fines docentes y no con el objetivo de optimizar el proceso. En el desarrollo y entrenamiento de modelos de ML (offline) es vital aprovecharnos de la vectorización para la optimización de recursos (operaciones en matrices o vectores completos en lugar de procesar los elementos individualmente mediante bucles). La vectorización es clave pero también dificulta el aprendizaje, ya que ofusca el código.

**Este Notebook no contiene ejercicios para entregar** pero es importante que dediqueis tiempo para entender los conceptos. 

## Referencias útiles para la práctica
1. API Matplotlib: [https://matplotlib.org/stable/api/index](https://matplotlib.org/stable/api/index)
2. API Numpy: [https://numpy.org/doc/stable/reference/](https://numpy.org/doc/stable/reference/)
3. Dataset para el ejercicio: [https://www.kaggle.com/datasets/camnugent/california-housing-prices](https://www.kaggle.com/datasets/camnugent/california-housing-prices).
4. [Curso](https://www.coursera.org/learn/machine-learning-course/home/welcome) de Machine Learning de Anxdrew NG en Coursera.
   * Este es un curso muy recomendable para el que quiera entender los conceptos básicos del ML.
   * La Unidad 03 es una adaptación de diferentes prácticas del curso a nuestro problema.


## Modelo de Regresión lineal de una variable

Un modelo de regresión lineal de una variable busca establecer una relación lineal entre dos variables: una variable independiente (input) y una variable dependiente (output).  El objetivo es encontrar la línea recta que mejor se ajuste a los datos, permitiendo predecir el valor de la variable dependiente a partir de la variable independiente.

La función del modelo que representa la regresión lineal simple se muestra a continuación:



$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$

Esta función tiene dos parámetros $w$ y $b$ y asigna un valor "y" (output) en función de una variable de entrada "x" (input). El ajuste de los parámetros $w$ y $b$ se realiza durante el proceso de entrenamiento del modelo.

## Conjunto de datos

Usaremos el conjunto de datos que hemos visto en las unidades anteriores, pero en este caso solo nos centraremos en 2 variables, para ajustarnos a las condiciones de la regresión lineal. En la Unidad 01 vimos que entre la variable a predecir *median_house_value* y *median_income* había una correlación lineal bastante alta. EL objetivo de esta Unidad será desarrollar una regresión lineal entre esas 2 variables.


**Recordad** que durante la Unidad 01 descubrimos que la *median_income* era una representación escalada del ingreso promedio para los hogares en una manzana, cuyo valor representaba decenas de miles de dolares (ej. 35=350.000$). Las 2 variables están en escalas muy diferentes. Para facilitar las gráficas y el seguimiento de la práctica, realizaremos un escalado sobre la característica *median_house_value* dividiendo su valor por 1.000.

**Nota**: aunque ya tenemos los datos preprocesados de la Unidad 02, vamos a trabajar con los datos originales para que su interpretación sea más sencilla. 




In [ ]:
try:
    import kagglehub
except ImportError as err:
    !pip install kagglehub
    import kagglehub
    
try:
    from rich import print
except ImportError as err:
    !pip install rich
    from rich import print

import pandas as pd
try:
    import pandas as pd
except ImportError as err:
    !pip install pandas
    import pandas as pd

try:
    from sklearn.model_selection import train_test_split
except ImportError as err:
    !pip install sklearn
    from sklearn.model_selection import train_test_split

try:
    import matplotlib.pyplot as plt
except ImportError as err:
    !pip install matplotlib
    import matplotlib.pyplot as plt

# Descarga del dataset
path = kagglehub.dataset_download("camnugent/california-housing-prices")
print("Path to dataset files:", path)
dataset=pd.read_csv(path+"/housing.csv")#Carga datos desde un CSV y devuelve un DataFrame 

#Generación de los datasets de entrenamiento y test
SEED=1234 #Semilla para asegurarnos que siempre se generen los mismos valores aleatorios
trainset, testset=train_test_split(dataset, test_size=0.3, train_size=0.7, random_state=SEED, shuffle=True, stratify=dataset["ocean_proximity"])

#Para esta Unidad solo nos interesan las variables "median_income" y "median_house_value"
trainset=trainset[["median_income","median_house_value" ]].copy()
testset=testset[["median_income","median_house_value" ]].copy()

#En unidades anteriores habóiamos descubierto que "median_house_value" tenía un límite superior
#que agrupaba todas las manzanas con precio medio de vivienda mayor que dicho límite
#para no condicionar el modelo, esas observaciones son eliminadas
#Eliminación de las observaciones con  median_house_value == max
#Eliminación de observaciones de Entrenamiento
trainset=trainset[trainset["median_house_value"]!=dataset["median_house_value"].max()]
#escalamos median_house_value en el mismo rango que median_income
trainset["median_house_value"]=trainset["median_house_value"].apply(lambda X: X/1000)
#Eliminación de observaciones de Test
testset=testset[testset["median_house_value"]!=dataset["median_house_value"].max()]


#Por motivos docentes y de visualización
#escalamos "median_house_value" en el mismo rango que "median_income"
#Ver *Nota* superior
testset["median_house_value"]=testset["median_house_value"].apply(lambda X: X/1000)


#Ploteamos los datos para visualizar la correlación entre las dos variables
trainset.plot(x="median_income", y="median_house_value", kind="scatter")
plt.show()
print(trainset.corr())







Empezaremos con lo más básico, cogiendo 2 puntos de los datos de entrenamiento para construir un modelo entre ellos. Vamos a trabajar desde la base, es decir, empleando [arrays](https://numpy.org/doc/stable/reference/generated/numpy.array.html) de numpy.

**Nota**: Las observaciones seleccionadas están deliberadamente escogidas por motivos docentes. Tenéis que tener en cuenta que este no es un problema que se pueda resolver con una regresión lineal de 1 variable, por lo que he buscado valores que ajusten correctamente.




In [ ]:
try:
    import numpy as np
except ImportError as err:
    !pip install numpy
    import numpy as np

# Las observaciones seleccionadas están deliberadamente escogidas por motivos docentes.
pto1=trainset.iloc[1697]#Devuelve una Serie de Pandas
pto2=trainset.iloc[4032] #Devuelve una Serie de Pandas

#Almacenamos en un array las entradas (inputs o 'x'), es decir, los valores de "median_income"
x_train = np.array([pto1.loc["median_income"],pto2.loc["median_income"]])

#Almacenamos en un array las salidas (outputs, labels, o 'y'), es decir, los valores de "median_house_value"
y_train = np.array([pto1.loc["median_house_value"],pto2.loc["median_house_value"]])

#Mostramos los puntos escogidos
print(f"x_train = {x_train}")
print(f"y_train = {y_train}")






#### Notación
- Usaremos $m$ para denotar el número de ejemplos que tenemos para entrenar, es decir, el número de observaciones de nuestro conjunto de entrenamiento (2 en este caso). Podemos emplear <code>len()</code> para obtener el valor o el método [shape()](https://numpy.org/doc/stable/reference/generated/numpy.shape.html) de los arrays de numpy.


- Usaremos la notación (x$^{(i)}$, y$^{(i)}$) para denotar la  $i$-esima observación del conjunto de entrenamiento.

    -    **Recordad** que en Python (y en Numpy) los índices empiezan en cero.


In [ ]:
print(f"Número de observaciones del entrenamiento: {len(x_train)}")
i = 0 # i-esima observación
x_i = x_train[i]
y_i = y_train[i]
print(f"La observación {i} es (x^({i}), y^({i})) = ({x_i}, {y_i})")

Visualicemos los datos de las observaciones seleccionadas en un gráfico. Veremos que estos puntos pueden encajar facilmente con una regresión lineal.

**Recordad** que los puntos fueron deliberadamente escogidos. Otros puntos del dataset de entrenamiento original no mostarían el mismo resultado.

In [ ]:
# Plotea los puntos
plt.figure()
plt.scatter(x_train, y_train, marker='x', c='r')
# Especifica el título
plt.title("Valor medio de la vivienda")

# Añade el eje y
plt.ylabel('median house value')
# Añade el eje x
plt.xlabel('median income')

plt.show()

## Modelo de regresión lineal de una variable

Recordamos la función del modelo para la regresión lineal de una sola variable:

$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$

Esta función solo permite representar lineas rectas (es la ecuación de una recta). Cambiando los valores $w$ y $b$ generamos diferentes tipos de recta, que se podrán ajustar mejor o peor a nuestros datos (el objetivo del entrenamiento es precisamente encontrar los mejores valores para $w$ y $b$). La clave es que no podemos escoger unos valores de $w$ y $b$ diferentes para cada observación, si no que necesitamos encontrar los valores que mejor se ajustan, de media, a todas las observaciones del conjunto de entrenamiento.


Si escogemos 2 valores cualquiera para  $w$ y $b$ podremos usar nuestro modelo para predecir la $y$ en base a las entradas $x$.

- Para $x^{(0)}$, `f_wb = w * x[0] + b`
- Para $x^{(1)}$, `f_wb = w * x[1] + b`




In [ ]:
def calcula_salida_modelo(x, w, b):
    m = len(x)
    print(m)
    predicciones = np.zeros(m)#genera una matriz de ceros con numpoy
    for i in range(m):
        #función del modelo que se aplica sobre los datos de entrada para unos valores w, b concretos
        predicciones[i] = w * x[i] + b 
    return predicciones


def plot_resultados_modelo(inputs,predicciones, labels):

    # Genera un plot con las predicciones
    plt.plot(inputs, predicciones, c='b',label='Predicciones')
    # Añade al plot los puntos reales de entrenamiento
    plt.scatter(inputs, labels, marker='x', c='r',label='Valores reales')

    
    # Añade un título al gráfico
    plt.title("Valor medio de la vivienda")
    # Añade el eje y
    plt.ylabel('median house value')
    # Añade el eje x
    plt.xlabel('median income')
    plt.show()


w = 100# seleccionamos un valor para w
b = 100# seleccionamos un valor para b 
print(f"w: {w}")
print(f"b: {b}")

predicciones = calcula_salida_modelo(x_train, w, b)
plot_resultados_modelo(x_train, predicciones,y_train)
for i, (x, y,pred) in enumerate(zip(x_train, y_train, predicciones)):
    print(f"x({i})={x} - valor real: {y} valor predicho: {pred:.2f}")


**Ejercicio**

Prueba a jugar con los valores de $w$ y $b$ para ver como varía la recta hasta conseguir unos valores que ajusten bien.



In [ ]:
#w = ?
#b = ?


predicciones = calcula_salida_modelo(x_train, w, b)
plot_resultados_modelo(x_train, predicciones,y_train)
for i, (x, y,pred) in enumerate(zip(x_train, y_train, predicciones)):
    print(f"x({i})={x} - valor real: {y} valor predicho: {pred:.2f}")

Una vez escogemos unos valores para los parámetros de nuestro modelo que ajustan bien con los datos de entrenamiento, podemos usar el modelo  para predecir con nuevos datos con los que nunca se entrenó.

In [ ]:
w = 200
b = 100

x_punto_test=3#probamos con un valor 

precio_medio_vivienda = w * x_punto_test + b 
precio_medio_vivienda*=1000 #lo devolvemos a la escala original

print(f"Precio de una vivienda en el que la familia tiene un ingreso de {x_punto_test*10000}$: {precio_medio_vivienda}$")

## Función de coste

Tenemos una función con 2 parámetros  ($w$ y $b$)  que debemos definir para ajustar dicho modelo pero, 
**¿Cómo escogemos los valores $w$ y $b$?**
$$ f_{w,b}(x^{(i)}) = wx^{(i)} + b \tag{1}$$


Obviamente no podemos estar probando con todas las posibles combinaciones "a mano". Necesitamos entrenar el modelo y que se ajusten automáticamente. Para poder decidir que una combinación es mejor que otra, es necesario poder establecer un **coste** de la combinación. De esta forma podremos comparar combinaciones y escoger entre las diferentes opciones. 


De forma general podemos representar la función de coste para los párametros $w$ y $b$ como: $J(w,b)$


Existen diferentes funciones de coste pero una habitual en ML es el *Mean Squared Error* (MSE).

$$MSE=\frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2\tag{2}$$

y particularmente, en problemas de regresión,  es habitual introducir un factor $\frac{1}{2}$, quedando la función de coste como: 

  $$J(w,b) = \frac{1}{2m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})^2 \tag{3}$$ 

 donde:
 -  $f_{w,b}(x^{(i)}) = wx^{(i)} + b $ (función del modelo).
 - $f_{w,b}(x^{(i)})$ es nuestra predicción para la entrada $x^i$ empleando unos parámetros ($w,b$) específicos. Por simplicidad es habitual representar la predicción como: $\hat{y}^i$
- $(f_{w,b}(x^{(i)}) -y^{(i)})^2$ es la diferencia al cuadrado entre el valor real y la predicción.   
- Todas esas diferencias a lo largo de las $m$ observaciones son sumadas y divididas por `2m` para producir el coste final, $J(w,b)$. 


**Fijaos** que el coste se calcula sobre todos las observaciones de entrenamiento, por lo que el objetivo es obtener los parámetros $w,b$ que mejor se comportan para todas las observaciones .

 **Importante**: Cuánto más cerca estén las predicciones de los valores reales, menor coste. **Debemos buscar una forma de minimizar la función de coste** para obtener la mejor solución.


$$\underset{w, b}{\text{minimize}}J(w,b)\tag{4}$$

In [ ]:
def calculo_coste(x, y, w, b): 
    """
    Calcula la función de coste para la regresión lineal
    
    Args:
      x (ndarray (m,)): Entradas, m observaciones 
      y (ndarray (m,)): salidas
      w,b (scalar)    : parámetros del modelo  
    
    Returns
        coste_total (float)
    """
    # observaciones
    m = len(x)
    cost_sum = 0 
    for i in range(m): #recorremos todas las observaciones y sumamos su coste
        pred = w * x[i] + b   
        cost_sum += (pred - y[i]) ** 2    
    return (1 / (2 * m)) * cost_sum #promediamos el coste 


#Comprobamos los diferentes costes que tendríamos para el mismo conjunto de entrada
#cuando varíamos los parámetros w, b
#Podéis probar con otras configuraciones
w_pruebas=[50,100,150]#posibles valores para w
b_pruebas=[50,100,150]#posibles valores para b

print("Para los datos de entrenamiento:")
for i,(x,y) in enumerate(zip(x_train, y_train)):
    print(f"(x^{i},y^{i})=({x},{y})")


for w, b in zip(w_pruebas,b_pruebas):
    print(f"El coste total con w={w} y b={b} es: {calculo_coste(x_train, y_train, w, b):.2f}")


### Visualización de la función de coste 

Para mejorar la comprensión de la función de coste y en que consiste su minimzación, vamos a simplificar el problema y suponer que el valor de $b$ es fijo y que solo tenemos que  buscar el valor óptimo de $w$, es decir, nuestra función solo tiene un parámetro "libre". Para cada posible valor de $w$, podemos calcular su función de coste ($J(w)$) y graficar dicho valor. A medida que vayamos calculando valores y graficándolos, iremos dibujando la curva del coste. 


Si solo tenemos 1 variable "libre" y le damos todos los posibles valores, la función de coste que podríamos pintar con los resultados tendría  forma convexa con un mínimo global. Nuestro objetivo minimizando dicha función de coste ($J(w)$) es encontrar dicho mínimo global.  Necesitamos un algoritmo que nos permita movernos por la función de coste hasta alcanzar el mínimo.

Veamos la representación gráfica de esta explicación. Fijaos en la figura inferior como evoluciona el coste (parte derecha) si movemos el valor de $w$ (parte izquierda). 
- La figura de la parte derecha refleja en azul la forma de la función de coste si graficásemos todos sus valores (es una aproximación).
- La figura de la parte derecha muestra el coste de una configuración concreta (valor de $w$) dentro de la función de coste. El objetivo es conseguir que el coste concreto se situe en el mínimo de la función.
- La figura de la parte izquierda muestra el coste de cada predicción (línea azul) respecto al valor real (puntos rojos). Cuanto más lejos el valor de la predicción del real, mayor coste.

**Nota**: Podéis jugar con el valor fijo de $b$.

**Nota 2**: el gráfico es interactivo, podéis modificar el valor de $w$. El refresco puede ser algo lento.

In [ ]:
%matplotlib widget
from lab_utils_uni import plt_intuition, plt_stationary, plt_update_onclick

#Esta función está en un fichero de Python con métodos útiles para la práctica
#No es necesario que la entendáis
#Se emplea solo por motivos docentes
plt_intuition(x_train,y_train, fixed_b=100)

Aumentemos la dificultad y dejemos "libres" los 2 parámetros $w$ y $b$. En este caso, para poder graficar la función de coste, necesitamos un gráfico en 3 dimensiones o emplear un gráfico de contornos (aka [isolineas](https://es.wikipedia.org/wiki/Isol%C3%ADnea)). 

- El gráfico de la derecha (gráfico de contornos) nos permite seleccionar una combinación de los parámetros $w$ y $b$
- El gráfico de la izquierda nos permite visualizar la recta y el coste de la selección de parámetros actual.



In [ ]:
plt.close('all') 
fig, ax, dyn_items = plt_stationary(x_train, y_train)
updater = plt_update_onclick(fig, ax, x_train, y_train, dyn_items)

## Gradiente descendente

Ahora que entendemos como funciona una regresión lineal y como se calcula el coste de los posibles valores ($w,b$), debemos encontrar una forma de buscarlos de forma automática. Esta claro que no podemos ir probando todas las posibles combinaciones de forma manual. 

¿Cómo minimizamos la función de coste? Empleando el algoritmo de **descenso de gradiente**. 

$$\underset{w, b}{\text{minimize}}J(w,b)\tag{4}$$


Este algoritmo define la forma de actualizar  los parámetros $w$, $b$.

$$\begin{align*} \text{Repetir}&\text{ hasta que converja:} \; \lbrace \newline
\;  w &= w -  \alpha \frac{\partial J(w,b)}{\partial w} \tag{5}  \; \newline 
 b &= b -  \alpha \frac{\partial J(w,b)}{\partial b}  \newline \rbrace
\end{align*}$$

donde:
- $\alpha$ es la tasa de aprendizaje o *learning rate*.
- $\frac{\partial J(w,b)}{\partial w}$ es la derivada parcial de la función de coste respecto a $w$.
- $\frac{\partial J(w,b)}{\partial b}$ es la derivada parcial de la función de coste respecto a $b$.

La actualización de parámetros implica la derivada parcial de la función de coste en función de $w$ y de $b$. **No es necesario aprendais a hacer la derivada parcial**, a continuación os proporciono el resultado de dicha derivada parcial:

$$
\begin{align}
\frac{\partial J(w,b)}{\partial w}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)})x^{(i)} \tag{4}\\
  \frac{\partial J(w,b)}{\partial b}  &= \frac{1}{m} \sum\limits_{i = 0}^{m-1} (f_{w,b}(x^{(i)}) - y^{(i)}) \tag{5}\\
\end{align}
$$


**Importante**: Los parámetros $w$, $b$ deben ser actualizados simultaneamente. Esto significa que las derivadas parciales deben calcularse antes de actualizar ninguno de los parámetros.

Os resumo los pasos del algoritmo: 
1. Inicializamos los valores de $w$ y $b$, típicamente con valores aleatorios.
2. Calculamos las predicciones.
3. Calculamos las derivadas parciales.
4. Ajustamos los parámetros $w$ y $b$ en función del gradiente.
5. Repetimos hasta cumplir la condición de salida.



El tamaño y dirección del "paso" para actualizar los parámetros $w$ y $b$ viene marcado por el resultado de la derivada parcial:
- **Dirección**: Si la derivada es positiva, debemos disminuir el parámetro; si es negativa, debemos aumentarlo.
- **Magnitud**: Qué tan grande es el cambio necesario. Valores mayores significan que el coste cambia rápidamente en esa dirección.

El hiperparámetro $\alpha$ (tasa de aprendizaje o *learnig rate*) permite aumentar o reducir el "paso" en la actualización de los parámetros. Este hiperparámetro es algo que deberemos escoger y ajustar al entrenar los modelos y que suele ser clave:
- Un $\alpha$ pequeño puede hacer que no lleguemos a un buen ajuste final del modelo en las iteraciones que le permitamos entrenar.
-  Un $\alpha$ demasiado grande puede resultar en un entrenamieno divergente. 


Vamos a implementar el descenso de gradiente para una sola variable con la idea de entender los fundamentos.


In [ ]:
def calculo_gradiente(x, y, w, b): 
    """
    Calcula el gradiente para la regresión lineal
    Args:
      x (ndarray (m,)): datos de entrada, m ejemplos 
      y (ndarray (m,)): valores de salida
      w,b (scalar)    : parámetros del modelo  
    Returns
      dj_dw (scalar): El gradiente del coste respecto a w
      dj_db (scalar): El gradiente del coste respecto a b     
     """
    
    # observaciones
    m = len(x)    

    #valores que devolveremos inicializados a cero
    dj_dw = 0
    dj_db = 0

    #calculo de los gradientes
    for i in range(m):  
        f_wb = w * x[i] + b #cálculo de la predicción
        dj_dw_i = (f_wb - y[i]) * x[i] 
        dj_db_i = f_wb - y[i] 
        dj_db += dj_db_i
        dj_dw += dj_dw_i 
    dj_dw = dj_dw / m 
    dj_db = dj_db / m 
        
    return dj_dw, dj_db

Para una mejor comprensión de los gradientes y la función de coste, vamos a fijar el valor de parámetro $b$ y probar con diferentes puntos para $w$. El gradiente nos devolverá:
- **Dirección**: Si la derivada es positiva, debemos disminuir el parámetro; si es negativa, debemos aumentarlo.
- **Magnitud**: Qué tan grande es el cambio necesario. Valores mayores significan que el coste cambia rápidamente en esa dirección.

**Nota**: En el mínimo global, la derivada será 0.

In [ ]:
from lab_utils_uni import plt_gradients
plt.close('all') 
plt_gradients(x_train,y_train, calculo_coste, calculo_gradiente, fixed_b=100, tested_points=[50,100,150,200, 300])
plt.show()

### Implementación del gradiente descendente.
Una vez comprendido como los gradientes nos ayudarán a saber la dirección y la magnitud de las actualizaciones de nuestros parámetros $w$ y $b$, ya solo necesitamos implementar el algoritmo del gradiente descendente para automatizar el proceso.

In [ ]:
def gradiente_descendente(x, y, w_in, b_in, alpha, num_iters, cost_function, gradient_function): 
    """
    Desarrolla el algoritmo de gradiente descendente para ajustar w y b. 
    Actualiza los valores w y b realizando num_iters cálculos de gradiente
    con una tasa de aprendizaje alpha
    
    Args:
      x (ndarray (m,))  : Datos entrada, m observacines 
      y (ndarray (m,))  : valores de salida
      w_in,b_in (scalar): Valores iniiales de w y b  
      alpha (float):     Learning rate o tasa de aprendizaje
      num_iters (int):   número de iteraciones (épocas) para ejecutar el gradiente descendente.
      cost_function:     función que calcula el coste
      gradient_function: función que calcula el radiente
      
    Returns:
      w (scalar): valor calculado de w una vez ejecutado el algoritmo de gradiente descendente.
      b (scalar): valor calculado de b una vez ejecutado el algoritmo de gradiente descendente.
      J_history (List): histórico de valores de coste 
      p_history (list): histórico de valores de (w,b)
      """
    
    # array para almancear los valores históricos de coste y (w,b)
    #Los usaremos para graficar
    J_history = []
    p_history = []
    b = b_in
    w = w_in
    
    for i in range(num_iters):
        # Calcula el gradiente empleando la función pasada como argumento
        dj_dw, dj_db = gradient_function(x, y, w , b)     

        # Actualiza los parámetros empleando la ecuación 5
        b = b - alpha * dj_db                            
        w = w - alpha * dj_dw                            

        ###Esta parte del algoritmo es opcional, es solo por histórico y visualización
        coste=cost_function(x, y, w , b)
        J_history.append(coste )#almacena el coste en el histórico
        p_history.append([w,b])#almacena los valores de w y b en el histórico

        #Mostramos el avance cada 10 iteraciones
        if i%10==0:
            print(f"Iteración {i}. Coste {coste}.",
                  f"dj_dw: {dj_dw: 0.2f}, dj_db: {dj_db: 0.2f}  ",
                  f"w: {w: 0.2f}, b:{b: 0.2f}"
                 )

 
    return w, b, J_history, p_history 

Probemos el algoritmo con nuestros datos:

**Nota**: si jugais con el número de iteraciones o con los valores de los parámetros de entrada, conseguiréis ajustar mejor el modelo.

In [ ]:
w_init=10#valores iniciales
b_init=10#valores iniciales
iteraciones=1000
alpha=0.01


w_calculado, b_calculado, J_history, p_history =gradiente_descendente(x_train, y_train, w_init, b_init, alpha, iteraciones, calculo_coste, calculo_gradiente)


Probemos el modelo con los resultados del algoritmo

In [ ]:
predicciones = calcula_salida_modelo(x_train, w_calculado, b_calculado)
plt.close('all') 
plot_resultados_modelo(x_train, predicciones,y_train)

### Coste vs iteraciones
Un gráfico del coste frente a las iteraciones es una medida ineresante para valorar el progreso en el descenso de gradiente. El coste siempre debería disminuir en ejecuciones exitosas. El cambio en el coste muy rápido al comienzo y luego tiende a desacelerar, por lo que, a menudo,  es útil graficar el descenso inicial en una escala diferente a la del descenso final. En los gráficos a continuación, observe la escala del coste en los ejes y el paso de iteración.

In [ ]:
# plot coste vs iteraciones versus iteration  
plt.close('all') 
fig, (ax1, ax2) = plt.subplots(1, 2, constrained_layout=True, figsize=(9,4))
ax1.plot(J_history[:100])



ax2.plot(100 + np.arange(len(J_history[100:])), J_history[100:])
ax1.set_title("Coste vs. iteración(incio)");  ax2.set_title("Coste vs. iteración (final)")
ax1.set_ylabel('Coste')            ;  ax2.set_ylabel('Coste') 
ax1.set_xlabel('Iteración')  ;  ax2.set_xlabel('Iteración') 
plt.show()

Podemos también visualizar la evolución de los parámetros $w$ y $b$ durante la ejecución del algoritmos de gradiente descendente respecto al gráfico de contornos. Veremos como la evolución de los valores se acerca, en cada iteración, al centro del gráfico, dónde se encuentra el mínimo global.

In [ ]:
from lab_utils_uni import plt_contour_wgrad
fig, ax = plt.subplots(1,1, figsize=(10, 6))
plt_contour_wgrad(x_train, y_train, p_history, ax)